This notebook is used to compare the scores of the PBs method in Brightway2 vs SimaPro.  
For that, the dataset of chemicals from the Green Chemistry paper will be used.

Selected dataset was solved in 
- SimaPro 9.1.0
- with ecoinvent v3.5 APOS system model
- PB-LCIA V0.72 (latest at that point)

In [1]:
import re

import brightway2 as bw
import numpy as np
import pandas as pd
from bw_recipe_2016 import add_recipe_2016
from project_path import ROOT_DIR

import productivity_boosters as boost
import aesa_pbs

In [2]:
print(f"productivity-boosters version: {boost.__version__}")
print(f"aesa_pbs version: {aesa_pbs.__version__}")

productivity-boosters version: 0.0.1.alpha1
aesa_pbs version: 0.9.0


## Prepare bw project

In [3]:
bw.projects

Brightway2 projects manager with 11 objects:
	aachen_fast_calculations
	check_PBsmethod
	default
	double_count_electricity
	premise_scenarios_ei35
	premisified_scenarios
	premisified_scenarios_cars
	spreadsheet
	supply chain graph
	test
	trial-SimaPro-import-DME-trucks
Use `projects.report()` to get a report on all projects.

In [4]:
# Create a "check_PBsmethod" project or activate it

if not [proj[0] for proj in bw.projects.report() if proj[0]=="check_PBsmethod"]:
    bw.projects.set_current("default")
    bw.projects.copy_project("check_PBsmethod", switch=True)
    
    # delete extra dbs
    del bw.databases['ei35cutoff']
    del bw.databases['ei38apos']
    del bw.databases['ei38cutoff']
else:
    bw.projects.set_current("check_PBsmethod")
print(f"Databases in '{bw.projects.current}' project: {bw.databases.list}")


# Delete "check_PBsmethod" project WHEN FINISHED TESTING!
# bw.projects.set_current("default")
# bw.projects.delete_project(name="check_PBsmethod", delete_dir=True)

# Delete database
# del bw.databases['A_technosphere_flows']

Databases in 'check_PBsmethod' project: ['A_technosphere_flows', 'biosphere3', 'ei35apos']


In [5]:
# bw.projects.dir

In [6]:
# Check installed AESA methods in project
installed_aesa_methods = [met for met in bw.methods if "aesa" in str(met).lower()]
print(
    f"{len(installed_aesa_methods)} methods installed, which correspond to {list({method[1] for method in installed_aesa_methods})} versions"
)

36 methods installed, which correspond to ['0.8.8+xlsxtoyaml.8.3', '0.9.0', '0.8.8'] versions


In [7]:
# Deregister old versions
# bw.Method(("some", "method")).deregister()

## Install LCIA methods

In [8]:
force_reinstall = False

if (
    any(
        "aesa" in str(aesa).lower() and str(aesa_pbs.__version__) in str(aesa)
        for aesa in bw.methods
    )
    and not force_reinstall
):
    print("AESA methods already present!")
else:
    aesa_pbs.add_aesa_pbs(verbose=False)

AESA methods already present!


In [9]:
# adapt db for N-direct fixation method
aesa_pbs.update_nitrogen_fertilizer_exchanges(aesa_pbs.get_nitrogenous_fertilizers("ei35apos"))

Updating activities that produce nitrogen fertilizer...
xx 'treatment, sludge from pulp and paper production, landfarming' (kilogram, Europe without Switzerland, None) already contains the necessary exchanges.
xx 'treatment, sludge from pulp and paper production, landfarming' (kilogram, RoW, None) already contains the necessary exchanges.
xx 'nutrient supply from coconut husk' (kilogram, GLO, None) already contains the necessary exchanges.
xx 'smelting and refining of nickel ore' (kilogram, GLO, None) already contains the necessary exchanges.
xx 'nutrient supply from poultry manure, fresh' (kilogram, GLO, None) already contains the necessary exchanges.
xx 'nutrient supply from poultry manure, dried' (kilogram, GLO, None) already contains the necessary exchanges.
xx 'nutrient supply from vinasse, from fermentation of sweet sorghum' (kilogram, GLO, None) already contains the necessary exchanges.
xx 'nutrient supply from compost' (kilogram, GLO, None) already contains the necessary exchan

In [10]:
if any(rcp for rcp in bw.methods if "recipe 2016" in str(rcp).lower()):
    print("ReCiPe2016 methods already present!")
else:
    add_recipe_2016()

ReCiPe2016 methods already present!


In [11]:
recipe2016 = [rcp for rcp in bw.methods if "recipe 2016" in str(rcp).lower()]
pbs = [aesa for aesa in bw.methods if "AESA" in str(aesa) and str(aesa_pbs.__version__) in str(aesa)]

## Map AESA methods

In [12]:
# convert bw method to tuple like ("method name", ["version"], ["categories"], "unit")
# format of the df_simapro.columns
def bw_methods_to_list_of_tuples(methods):
    to_return = []
    for method in methods:
        to_return.append(method + (bw.Method(method).metadata["unit"],))
    return to_return

In [13]:
pbs_tuples = bw_methods_to_list_of_tuples(pbs)

In [14]:
aesa_method_mapper = {
    "('PBs-LCIA (baseline) V0.72', 'Climate change - CO2 concentration', 'ppm')": [
        "climate change",
        "co2 concentration",
    ],
    "('PBs-LCIA (baseline) V0.72', 'Climate change - Energy imbalance', 'Wm-2')": [
        "climate change",
        "energy imbalance",
    ],
    "('PBs-LCIA (baseline) V0.72', 'Stratospheric ozone depletion', 'DU')": [
        "ozone depletion"
    ],
    "('PBs-LCIA (baseline) V0.72', 'Ocean acidification', 'Omega Aragon')": [
        "ocean acidification"
    ],
    "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - P', 'Tg P')": [
        "biogeochemical flows",
        "phosphorus",
    ],
    "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - N', 'Tg N')": [
        "biogeochemical flows",
        "nitrogen",
        "inverse",  # inverse modelling
    ],
    "('PBs-LCIA (baseline) V0.72', 'Land-system change - Global', '%')": [
        "land-system change",
        "global",
    ],
    "('PBs-LCIA (baseline) V0.72', 'Freshwater use - Global', 'km3')": [
        "freshwater use",
        "global",
    ],
    "('PBs-LCIA (baseline) V0.72', 'Change in biosphere integrity - BII loss', '% BII loss')": [
        "change in biosphere integrity",
        "total",  # functional diversity, total
    ],
}


def sp_to_bw_method_mapper(bw_methods_tuples, template_mapper):
    "Return sp to bw methods map"
    map_methods = dict()
    for method in bw_methods_tuples:
        map_methods[method] = str(method) + " (not in sp)"
        for key, lst in template_mapper.items():
            if all(item in str(method).lower() for item in lst):
                map_methods[method] = key

    return {v: k for k, v in map_methods.items()}

In [15]:
sp_to_bw_map_pbs = sp_to_bw_method_mapper(pbs_tuples, aesa_method_mapper)
# sp_to_bw_map_pbs

## Read set of SimaPro solved chemicals

In [16]:
SIMAPRO_DATA = Path(ROOT_DIR + r"\data\raw\lcia-results-from-sp910-combined.xlsx")

with pd.ExcelFile(SIMAPRO_DATA, engine=None) as xl_obj:
    df_simapro_raw = pd.read_excel(xl_obj, sheet_name="Sheet1")

# df_simapro_raw.head(3)

# ReCiPe methods in dataframe
# [i for i in df_simapro_raw.columns if "ReCiPe 2016 Midpoint" in i]

# AESA methods in dataframe
# [i for i in df_simapro_raw.columns if "0.72" in i]

df_simapro = df_simapro_raw[
    ["Activity"] 
    + [i for i in df_simapro_raw.columns if "0.72" in i]
    + [i for i in df_simapro_raw.columns if "ReCiPe 2016 Midpoint" in i]
]
df_simapro.head(3)

# df_simapro.columns

,Activity,"('PBs-LCIA (baseline) V0.72', 'Climate change - CO2 concentration', 'ppm')","('PBs-LCIA (baseline) V0.72', 'Climate change - Energy imbalance', 'Wm-2')","('PBs-LCIA (baseline) V0.72', 'Stratospheric ozone depletion', 'DU')","('PBs-LCIA (baseline) V0.72', 'Ocean acidification', 'Omega Aragon')","('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - P', 'Tg P')","('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - N', 'Tg N')","('PBs-LCIA (baseline) V0.72', 'Land-system change - Global', '%')","('PBs-LCIA (baseline) V0.72', 'Freshwater use - Global', 'km3')","('PBs-LCIA (baseline) V0.72', 'Change in biosphere integrity - BII loss', '% BII loss')",...,"('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Marine eutrophication', 'kg N eq')","('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial ecotoxicity', 'kg 1,4-DCB')","('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Freshwater ecotoxicity', 'kg 1,4-DCB')","('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Marine ecotoxicity', 'kg 1,4-DCB')","('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Human carcinogenic toxicity', 'kg 1,4-DCB')","('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Human non-carcinogenic toxicity', 'kg 1,4-DCB')","('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Land use', 'm2a crop eq')","('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Mineral resource scarcity', 'kg Cu eq')","('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Fossil resource scarcity', 'kg oil eq')","('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Water consumption', 'm3')"
0,Sulfuric acid {RoW}| market for sulfuric acid ...,3.687066e-12,4.971751e-14,1.156094e-15,1.126669e-14,2.047673e-16,3.232508e-14,1.099348e-16,7.588386e-13,4.280597e-14,...,0.000007,4.032473,0.036113,0.052831,0.014920,1.402386,0.007821,0.004239,0.138144,0.020295
1,Sulfuric acid {RER}| market for sulfuric acid ...,2.685277e-12,3.625857e-14,8.425343e-16,8.205477e-15,2.162822e-16,3.982650e-14,1.233763e-16,2.679813e-13,3.340084e-14,...,0.000006,2.906612,0.012614,0.019320,0.009662,0.514523,0.006503,0.001581,0.157021,0.025921
2,Sulfamic acid {GLO}| market for sulfamic acid ...,5.443214e-11,7.277248e-13,8.913290e-15,1.663306e-13,1.122301e-15,2.008279e-13,5.993186e-16,4.956729e-12,5.046367e-13,...,0.002215,13.651391,0.092072,0.135565,0.075901,3.221187,0.037790,0.011219,0.786975,0.095634


```python
# all ReCiPe2016 categories implemented in bw_recipe_2016
# use them to update `map_methods`
[
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Mineral Resource Scarcity", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Mineral Resource Scarcity", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Mineral Resource Scarcity", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Fossil resource scarcity"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ozone Formation", "Damage to Ecosystems"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Marine eutrophication"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Terrestrial Acidification"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Global Warming", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Global Warming", "1000 year timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ionizing Radiation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ionizing Radiation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ionizing Radiation", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ozone Formation", "Damage to Humans"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Stratospheric Ozone Depletion", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Stratospheric Ozone Depletion", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Stratospheric Ozone Depletion", "Infinite timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Marine", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Marine", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Marine", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Terrestrial", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Terrestrial", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Terrestrial", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Land occupation"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Freshwater Eutrophication"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Carcinogenic", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Freshwater", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Freshwater", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Freshwater", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Water consumption"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Particulate Matter Formation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Particulate Matter Formation", "Hierarchist", ),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Particulate Matter Formation", "Egalitarian", ),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Land transformation"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Non-carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Non-carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Non-carcinogenic", "Egalitarian"),


("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Global Warming", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Global Warming", "1000 year timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Stratospheric Ozone Depletion", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Stratospheric Ozone Depletion", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Stratospheric Ozone Depletion", "Infinite timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ionizing Radiation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ionizing Radiation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ionizing Radiation", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Particulate Matter Formation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Particulate Matter Formation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Particulate Matter Formation", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ozone Formation", "Damage to Humans", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ozone Formation", "Damage to Humans", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ozone Formation", "Damage to Humans", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Carcinogenic", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Non-carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Non-carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Non-carcinogenic", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Water consumption", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Water consumption", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Water consumption", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Global Warming", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Global Warming", "1000 year timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ozone Formation", "Damage to Ecosystems", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ozone Formation", "Damage to Ecosystems", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ozone Formation", "Damage to Ecosystems", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Terrestrial Acidification", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Terrestrial Acidification", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Terrestrial Acidification", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ecotoxicity", "Terrestrial", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ecotoxicity", "Terrestrial", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ecotoxicity", "Terrestrial", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Water consumption", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Water consumption", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Water consumption", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land occupation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land occupation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land occupation", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Global Warming", "100 year timescale","Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Global Warming", "1000 year timescale","Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Freshwater Eutrophication", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Freshwater Eutrophication", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Freshwater Eutrophication", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Ecotoxicity", "Freshwater", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Ecotoxicity", "Freshwater", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Ecotoxicity", "Freshwater", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Water consumption", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Water consumption", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Water consumption", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Ecotoxicity", "Marine", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Ecotoxicity", "Marine", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Ecotoxicity", "Marine","Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Marine eutrophication", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Marine eutrophication", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Marine eutrophication", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Mineral Resource Scarcity", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Mineral Resource Scarcity", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Mineral Resource Scarcity", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land transformation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land transformation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land transformation", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Fossil resource scarcity", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Fossil resource scarcity", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Fossil resource scarcity", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Weighted single score", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Weighted single score", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Weighted single score", "Aggregated", "Individualist"),
]
```

## Rename columns in dataframe

In [17]:
sp_columns_map = sp_to_bw_map_pbs.copy()

gwp_recipe = bw_methods_to_list_of_tuples(
    [
        m
        for m in recipe2016
        if all(
            i in str(m).lower() for i in ["midpoint", "global warming", "hierarchist"]
        )
    ]
)[0]

# add "Activity" and "GWP ReCiPe" columns
sp_columns_map.update(
    {
        "Activity": "Activity",
        "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Global warming', 'kg CO2 eq')": gwp_recipe,
    }
)

In [18]:
empty_columns = [col for col in sp_columns_map.keys() if col not in df_simapro.columns]
len(empty_columns)

3

In [19]:
df_simapro_copy = df_simapro.copy()
for col in empty_columns:
    df_simapro_copy[col] = np.nan
# df_simapro_copy.columns

df_simapro_copy = df_simapro_copy[sp_columns_map.keys()]
df_sp_scores = df_simapro_copy.rename(columns=sp_columns_map)
df_sp_scores.head(3)

,"(AESA (PBs-LCIA), 0.9.0, biogeochemical flows, phosphorus, Tg P)","(AESA (PBs-LCIA), 0.9.0, land-system change, global, % forested land)","(AESA (PBs-LCIA), 0.9.0, biogeochemical flows, nitrogen, directly fixated, Tg N)","(AESA (PBs-LCIA), 0.9.0, freshwater use, global, km3)","(AESA (PBs-LCIA), 0.9.0, ozone depletion, stratospheric O3 concentration, Dobson unit)","(AESA (PBs-LCIA), 0.9.0, climate change, atmospheric CO2 concentration, ppm)","(AESA (PBs-LCIA), 0.9.0, change in biosphere integrity, functional diversity, total, % BII loss)","(AESA (PBs-LCIA), 0.9.0, change in biosphere integrity, functional diversity, direct land use, % BII loss)","(AESA (PBs-LCIA), 0.9.0, change in biosphere integrity, functional diversity, CO2eq emissions, % BII loss)","(AESA (PBs-LCIA), 0.9.0, biogeochemical flows, nitrogen, inverse modelling, surface water, Tg N)","(AESA (PBs-LCIA), 0.9.0, climate change, energy imbalance at top-of-atmosphere, Wm-2)","(AESA (PBs-LCIA), 0.9.0, ocean acidification, carbonate ion concentration, omega aragonite)",Activity,"(ReCiPe 2016, 1.1 (20180117), Midpoint, Global Warming, 100 year timescale, Hierarchist, kg CO2eq/kg GHG)"
0,2.047673e-16,1.099348e-16,NaN,7.588386e-13,1.156094e-15,3.687066e-12,4.280597e-14,NaN,NaN,3.232508e-14,4.971751e-14,1.126669e-14,Sulfuric acid {RoW}| market for sulfuric acid ...,0.160737
1,2.162822e-16,1.233763e-16,NaN,2.679813e-13,8.425343e-16,2.685277e-12,3.340084e-14,NaN,NaN,3.982650e-14,3.625857e-14,8.205477e-15,Sulfuric acid {RER}| market for sulfuric acid ...,0.123240
2,1.122301e-15,5.993186e-16,NaN,4.956729e-12,8.913290e-15,5.443214e-11,5.046367e-13,NaN,NaN,2.008279e-13,7.277248e-13,1.663306e-13,Sulfamic acid {GLO}| market for sulfamic acid ...,2.202389


In [20]:
len(list(df_sp_scores.Activity))

947

In [21]:
# list(df_simapro.Activity)

## Match sp activities to bw

In [22]:
pattern = re.compile(r"(^.*) {(.*)}\| (.*) \|")

activities_sp = {}
for i in list(df_sp_scores.Activity):
    mo = re.match(pattern, i)
    if mo:
        activities_sp[i] = {
            "reference product": mo[1].lower(),
            "name": mo[3].lower(),
            "location": mo[2],
        }
# activities_sp

In [23]:
eidb = bw.Database("ei35apos")
# eidb

eidb_markets = [activity for activity in eidb if "market" in activity["name"]]

activities_bw = []

for act_sp in boost.progressbar(
    itobj=activities_sp.keys(),
    prefix="Progress: ",
    size=20,
    unit="activity(ies)",
):
    # activities_sp.setdefault("spname", None)
    for act in eidb_markets:
        if (
            activities_sp[act_sp]["reference product"] == str(act["reference product"]).lower()
            and "market" in str(act["name"])
            and activities_sp[act_sp]["location"] == str(act["location"])
        ):
            activities_bw.append(act)
            activities_sp[act_sp].update({"einame": act})


Progress: [🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩] 947/947 activity(ies)
Duration: 0:00:05.132830


In [24]:
# activities not returned to activities_sp!!!
set(activities_bw)- set([b["einame"] for b in activities_sp.values() if b.get("einame")])

{'market for neutralising agent, sodium hydroxide-equivalent' (kilogram, GLO, None),
 'fraction 1 from naphtha separation to generic market for chemical, organic' (kilogram, GLO, None),
 'fraction 8 from naphtha separation to generic market for chemical, organic' (kilogram, GLO, None),
 'methylcyclopentane to generic market for solvent, organic' (kilogram, GLO, None),
 'cottonseed oil, refined, to generic market for vegetable oil, refined' (kilogram, GLO, None),
 '1,1-dimethylcyclopentane to generic market for solvent, organic' (kilogram, GLO, None),
 'white spirit to generic market for solvent for paint' (kilogram, GLO, None),
 'praseodymium oxide to generic market for mischmetal' (kilogram, GLO, None),
 'tin dioxide to generic market for polishing powder' (kilogram, GLO, None),
 'market for vegetable oil, refined' (kilogram, GLO, None),
 'market for soil pH raising agent, as CaCO3' (kilogram, GLO, None),
 'ethanol, from fermentation, to niche market for ethanol, at service station' (

In [25]:
len(activities_bw)

978

In [26]:
len(activities_sp)

947

In [27]:
# drop unmatched sp activities
to_drop = [i for i in activities_sp if activities_sp[i].get("einame", None)==None]

df_sp_scores = df_sp_scores[~df_sp_scores.Activity.isin(to_drop)]
df_sp_scores.shape

(945, 14)

In [28]:
def get_ei_activity(key):
    act = activities_sp[key].get("einame", None)#["activity"]
    # return (act["reference product"], act["location"], act["name"], act["activity"])
    return act

In [29]:
df_sp_scores['einame'] = df_sp_scores['Activity'].apply(lambda key: get_ei_activity(key))
df_sp_scores["database"] = df_sp_scores['Activity'].apply(lambda key: get_ei_activity(key)["database"])
df_sp_scores["activity name"] = df_sp_scores['Activity'].apply(lambda key: get_ei_activity(key)["name"]) 
df_sp_scores["reference product"] = df_sp_scores['Activity'].apply(lambda key: get_ei_activity(key)["reference product"]) 
df_sp_scores["unit"] = df_sp_scores['Activity'].apply(lambda key: get_ei_activity(key)["unit"]) 
df_sp_scores["location"] = df_sp_scores['Activity'].apply(lambda key: get_ei_activity(key)["location"]) 
df_sp_scores.head(3)

,"(AESA (PBs-LCIA), 0.9.0, biogeochemical flows, phosphorus, Tg P)","(AESA (PBs-LCIA), 0.9.0, land-system change, global, % forested land)","(AESA (PBs-LCIA), 0.9.0, biogeochemical flows, nitrogen, directly fixated, Tg N)","(AESA (PBs-LCIA), 0.9.0, freshwater use, global, km3)","(AESA (PBs-LCIA), 0.9.0, ozone depletion, stratospheric O3 concentration, Dobson unit)","(AESA (PBs-LCIA), 0.9.0, climate change, atmospheric CO2 concentration, ppm)","(AESA (PBs-LCIA), 0.9.0, change in biosphere integrity, functional diversity, total, % BII loss)","(AESA (PBs-LCIA), 0.9.0, change in biosphere integrity, functional diversity, direct land use, % BII loss)","(AESA (PBs-LCIA), 0.9.0, change in biosphere integrity, functional diversity, CO2eq emissions, % BII loss)","(AESA (PBs-LCIA), 0.9.0, biogeochemical flows, nitrogen, inverse modelling, surface water, Tg N)","(AESA (PBs-LCIA), 0.9.0, climate change, energy imbalance at top-of-atmosphere, Wm-2)","(AESA (PBs-LCIA), 0.9.0, ocean acidification, carbonate ion concentration, omega aragonite)",Activity,"(ReCiPe 2016, 1.1 (20180117), Midpoint, Global Warming, 100 year timescale, Hierarchist, kg CO2eq/kg GHG)",einame,database,activity name,reference product,unit,location
0,2.047673e-16,1.099348e-16,NaN,7.588386e-13,1.156094e-15,3.687066e-12,4.280597e-14,NaN,NaN,3.232508e-14,4.971751e-14,1.126669e-14,Sulfuric acid {RoW}| market for sulfuric acid ...,0.160737,"[comment, classifications, activity type, acti...",ei35apos,market for sulfuric acid,sulfuric acid,kilogram,RoW
1,2.162822e-16,1.233763e-16,NaN,2.679813e-13,8.425343e-16,2.685277e-12,3.340084e-14,NaN,NaN,3.982650e-14,3.625857e-14,8.205477e-15,Sulfuric acid {RER}| market for sulfuric acid ...,0.123240,"[comment, classifications, activity type, acti...",ei35apos,market for sulfuric acid,sulfuric acid,kilogram,RER
2,1.122301e-15,5.993186e-16,NaN,4.956729e-12,8.913290e-15,5.443214e-11,5.046367e-13,NaN,NaN,2.008279e-13,7.277248e-13,1.663306e-13,Sulfamic acid {GLO}| market for sulfamic acid ...,2.202389,"[comment, classifications, activity type, acti...",ei35apos,market for sulfamic acid,sulfamic acid,kilogram,GLO


In [30]:
def rename_column(col):
    if isinstance(col, tuple):
        return ' - '.join(col[:-1])+f" ({col[-1]})"
    else:
        return col

df_sp_scores = df_sp_scores.rename(columns=rename_column)
df_sp_scores.columns

Index(['AESA (PBs-LCIA) - 0.9.0 - biogeochemical flows - phosphorus (Tg P)',
       'AESA (PBs-LCIA) - 0.9.0 - land-system change - global (% forested land)',
       'AESA (PBs-LCIA) - 0.9.0 - biogeochemical flows - nitrogen - directly fixated (Tg N)',
       'AESA (PBs-LCIA) - 0.9.0 - freshwater use - global (km3)',
       'AESA (PBs-LCIA) - 0.9.0 - ozone depletion - stratospheric O3 concentration (Dobson unit)',
       'AESA (PBs-LCIA) - 0.9.0 - climate change - atmospheric CO2 concentration (ppm)',
       'AESA (PBs-LCIA) - 0.9.0 - change in biosphere integrity - functional diversity - total (% BII loss)',
       'AESA (PBs-LCIA) - 0.9.0 - change in biosphere integrity - functional diversity - direct land use (% BII loss)',
       'AESA (PBs-LCIA) - 0.9.0 - change in biosphere integrity - functional diversity - CO2eq emissions (% BII loss)',
       'AESA (PBs-LCIA) - 0.9.0 - biogeochemical flows - nitrogen - inverse modelling, surface water (Tg N)',
       'AESA (PBs-LCIA) - 0.9.0 -

In [31]:
dct_FUs = [{act: 1} for act in df_sp_scores.einame] # if isinstance(act, bd.backends.peewee.proxies.Activity)]
# dct_FUs = [{act: 1} for act in activities_bw]
# dct_FUs

In [32]:
selected_methods = pbs + [gwp_recipe[:-1]]
# selected_methods

In [33]:
pd.options.display.float_format = '{:12.5e}'.format

In [34]:
bw.calculation_setups

CalculationSetups dictionary with 1 object(s):
	chemicals

In [35]:
df_bw_scores = boost.pretty_lca(FUs=dct_FUs, methods=selected_methods, setup_name="chemicals")
df_bw_scores.head(3)

Duration: 0:00:34.170071


,database,activity name,reference product,unit,location,AESA (PBs-LCIA) - 0.9.0 - biogeochemical flows - phosphorus (Tg P),AESA (PBs-LCIA) - 0.9.0 - land-system change - global (% forested land),AESA (PBs-LCIA) - 0.9.0 - biogeochemical flows - nitrogen - directly fixated (Tg N),AESA (PBs-LCIA) - 0.9.0 - freshwater use - global (km3),AESA (PBs-LCIA) - 0.9.0 - ozone depletion - stratospheric O3 concentration (Dobson unit),AESA (PBs-LCIA) - 0.9.0 - climate change - atmospheric CO2 concentration (ppm),AESA (PBs-LCIA) - 0.9.0 - change in biosphere integrity - functional diversity - total (% BII loss),AESA (PBs-LCIA) - 0.9.0 - change in biosphere integrity - functional diversity - direct land use (% BII loss),AESA (PBs-LCIA) - 0.9.0 - change in biosphere integrity - functional diversity - CO2eq emissions (% BII loss),"AESA (PBs-LCIA) - 0.9.0 - biogeochemical flows - nitrogen - inverse modelling, surface water (Tg N)",AESA (PBs-LCIA) - 0.9.0 - climate change - energy imbalance at top-of-atmosphere (Wm-2),AESA (PBs-LCIA) - 0.9.0 - ocean acidification - carbonate ion concentration (omega aragonite),ReCiPe 2016 - 1.1 (20180117) - Midpoint - Global Warming - 100 year timescale - Hierarchist (kg CO2eq/kg GHG)
0,ei35apos,market for sulfuric acid,sulfuric acid,kilogram,RoW,2.04767e-16,1.09935e-16,1.01678e-13,2.02947e-11,1.15609e-15,3.68707e-12,4.59113e-14,1.25277e-14,3.33836e-14,4.67079e-14,4.97176e-14,1.12667e-14,1.60736e-01
1,ei35apos,market for sulfuric acid,sulfuric acid,kilogram,RER,2.16282e-16,1.23376e-16,5.01396e-14,2.59210e-11,8.42535e-16,2.68528e-12,3.62595e-14,1.06638e-14,2.55958e-14,4.97176e-14,3.62586e-14,8.20548e-15,1.23239e-01
2,ei35apos,market for sulfamic acid,sulfamic acid,kilogram,GLO,1.12230e-15,5.99318e-16,3.25880e-10,9.56326e-11,8.91329e-15,5.44322e-11,5.21511e-13,6.40927e-14,4.57418e-13,1.29056e-12,7.27725e-13,1.66331e-13,2.20238e+00


In [36]:
df_bw_scores.shape

(945, 18)

In [37]:
# np.isclose(7.40056E-13*10e13,1.05827e-13*10e13)

# results

In [38]:
if "Activity" in df_sp_scores.columns:
    df_sp_scores = df_sp_scores.drop(columns=["Activity", "einame"])
    df_sp_scores = df_sp_scores.set_index(["database", "activity name", "reference product", "unit", "location"])
df_sp_scores.head(2)

AESA (PBs-LCIA) - 0.9.0 - biogeochemical flows - phosphorus (Tg P)  \
database activity name            reference product unit     location                                                                       
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             2.04767e-16                    
                                                             RER                                             2.16282e-16                    

                                                                       AESA (PBs-LCIA) - 0.9.0 - land-system change - global (% forested land)  \
database activity name            reference product unit     location                                                                            
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             1.09935e-16                         
                                                             RER                                             1.23376e-16                         

                                                                       AESA (PBs-LCIA) - 0.9.0 - biogeochemical flows - nitrogen - directly fixated (Tg N)  \
database activity name            reference product unit     location                                                                                        
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                                     NaN                                     
                                                             RER                                                     NaN                                     

                                                                       AESA (PBs-LCIA) - 0.9.0 - freshwater use - global (km3)  \
database activity name            reference product unit     location                                                            
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             7.58839e-13         
                                                             RER                                             2.67981e-13         

                                                                       AESA (PBs-LCIA) - 0.9.0 - ozone depletion - stratospheric O3 concentration (Dobson unit)  \
database activity name            reference product unit     location                                                                                             
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             1.15609e-15                                          
                                                             RER                                             8.42534e-16                                          

                                                                       AESA (PBs-LCIA) - 0.9.0 - climate change - atmospheric CO2 concentration (ppm)  \
database activity name            reference product unit     location                                                                                   
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             3.68707e-12                                
                                                             RER                                             2.68528e-12                                

                                                                       AESA (PBs-LCIA) - 0.9.0 - change in biosphere integrity - functional diversity - total (% BII loss)  \
database activity name            reference product unit     location                                                                                                        
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             4.28060e-14                                 

In [39]:
if "database" in df_bw_scores.columns:
    df_bw_scores = df_bw_scores.set_index(["database", "activity name", "reference product", "unit", "location"])
df_bw_scores.head(2) 

AESA (PBs-LCIA) - 0.9.0 - biogeochemical flows - phosphorus (Tg P)  \
database activity name            reference product unit     location                                                                       
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             2.04767e-16                    
                                                             RER                                             2.16282e-16                    

                                                                       AESA (PBs-LCIA) - 0.9.0 - land-system change - global (% forested land)  \
database activity name            reference product unit     location                                                                            
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             1.09935e-16                         
                                                             RER                                             1.23376e-16                         

                                                                       AESA (PBs-LCIA) - 0.9.0 - biogeochemical flows - nitrogen - directly fixated (Tg N)  \
database activity name            reference product unit     location                                                                                        
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             1.01678e-13                                     
                                                             RER                                             5.01396e-14                                     

                                                                       AESA (PBs-LCIA) - 0.9.0 - freshwater use - global (km3)  \
database activity name            reference product unit     location                                                            
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             2.02947e-11         
                                                             RER                                             2.59210e-11         

                                                                       AESA (PBs-LCIA) - 0.9.0 - ozone depletion - stratospheric O3 concentration (Dobson unit)  \
database activity name            reference product unit     location                                                                                             
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             1.15609e-15                                          
                                                             RER                                             8.42535e-16                                          

                                                                       AESA (PBs-LCIA) - 0.9.0 - climate change - atmospheric CO2 concentration (ppm)  \
database activity name            reference product unit     location                                                                                   
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             3.68707e-12                                
                                                             RER                                             2.68528e-12                                

                                                                       AESA (PBs-LCIA) - 0.9.0 - change in biosphere integrity - functional diversity - total (% BII loss)  \
database activity name            reference product unit     location                                                                                                        
ei35apos market for sulfuric acid sulfuric acid     kilogram RoW                                             4.59113e-14                                 

In [40]:
# relative difference simapro vs brightway2 scores
# returns 
# if > 0 -- % increase of bw score compared to sp score
# if < 0 -- % decrease of bw score compared to sp score
res = (df_bw_scores-df_sp_scores).div(df_sp_scores)
# res

In [41]:
def make_pretty(styler):
    styler.background_gradient(axis=None, vmin=-100, vmax=100, cmap="seismic")
    styler.format('{:.2%}')
    return styler

In [42]:
# big diffrences in: 
# "market for green manure" due to diffenent units, bw -> ha, simapro -> m2
# lime to generic market for soil pH raising agent: recalculated:OK
# methyl acetate to generic market for solvent, organic', 'GLO: recalculated: OK

In [43]:
res.style.pipe(make_pretty)

# one-by-one comparisons